In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
top_dir = Path.cwd().parent.parent

sheets = pd.read_excel(top_dir / "data" / "gorilla_test_data.xlsx", sheet_name=None)
for k, v in sheets.items():
    print(f"{k}:  {v.columns=}")
    


In [ ]:
meter_list = sheets["meter_list"]
forecast_table = sheets["forecast_table"]
rate_table = sheets["rate_table"]

`meter_list` and `forecast_table` together will give the unique information needed to look up the rate in rate_table. Merge `meter_list` and `forecast_table` on `meter_id`.

In [ ]:
merged_ml_ft = meter_list.merge(forecast_table, on="meter_id", how="right")
merged_ml_ft

Merge with `rate_table` to be able to perform vectorized operations.

In [ ]:
merged_ml_ft_rt = merged_ml_ft.merge(rate_table, on=["exit_zone","date"], how="inner")
merged_ml_ft_rt

Set `NaN` values in `aq_max_kwh` column to `np.inf` value.

In [ ]:
merged_ml_ft_rt["aq_max_kwh"] = merged_ml_ft_rt["aq_max_kwh"].fillna(value=np.inf)
merged_ml_ft_rt

Remove all entries where for `rate_p_per_kwh` following condition isn't fulfilled: `aq_kwh` >= `aq_min_kwh` and `aq_kwh` < `aq_max_kwh`. Special case where `aq_max_kwh` == np.nan. This means the upper bound is infinite.

In [ ]:
merged_ml_ft_rt = merged_ml_ft_rt[merged_ml_ft_rt["rate_p_per_kwh"].between(merged_ml_ft_rt["aq_min_kwh"], merged_ml_ft_rt["aq_max_kwh"], inclusive="left")]
merged_ml_ft_rt

Calculate cost per day (`kwh` * `rate_p_per_kwh`).

In [ ]:
merged_ml_ft_rt["cost_per_day"] = merged_ml_ft_rt["kwh"] * merged_ml_ft_rt["rate_p_per_kwh"]
merged_ml_ft_rt

Aggregating `cost_per_day` and `kwh`. Summing.

In [ ]:
total_cost = merged_ml_ft_rt[["meter_id", "cost_per_day"]].groupby("meter_id").sum()
total_cost["cost_per_day (p)"] = total_cost["cost_per_day"] / 100
total_consumption = merged_ml_ft_rt[["meter_id", "kwh"]].groupby("meter_id").sum()
merged_tc_tc = total_cost.merge(total_consumption, on="meter_id").reset_index()
merged_tc_tc

Reformatting column names and dropping `cost_per_day`.

In [ ]:
solution1 = merged_tc_tc.drop("cost_per_day", axis=1).rename(columns={"cost_per_day (p)": "Total Cost (£)", "meter_id": "Meter ID", "kwh": "Total Estimated Consumption (kWh)"}).round(2)

**SOLUTION 1**: Calculate total cost per meter (pounds) and total consumption (kwh).

In [ ]:
solution1